# Compare DNN Model
- Compare Keras vs onnx runtime

## Make DNN Model and Convert ONNX
- Keras로 DNN 모델 생성 
- 학습은 Mnist Data로 진행
- <b> .h5 모델 로드 후 .onnx로 변환 </b>


In [1]:
# Imports
import tensorflow
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

import os
os.environ['TF_KERAS'] = '1'

# Configuration options
feature_vector_length = 784
num_classes = 10

# Load the data
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()

# Reshape the data - MLPs do not understand such things as '2D'.
# Reshape to 28 x 28 pixels = 784 features
X_train = X_train.reshape(X_train.shape[0], feature_vector_length)
X_test = X_test.reshape(X_test.shape[0], feature_vector_length)

# Convert into greyscale
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

# Convert target classes to categorical ones
Y_train = to_categorical(Y_train, num_classes)
Y_test = to_categorical(Y_test, num_classes)

# Set the input shape
input_shape = (feature_vector_length,)
print(f'Feature shape: {input_shape}')

# Create the model
model = Sequential()
model.add(Dense(350, input_shape=input_shape, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# Configure the model and start training
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, Y_train, epochs=1, batch_size=2000, verbose=1, validation_split=0.2)

model.save('model/DNN_MLP_basic.h5')

# Test the model after training
# test_results = model.evaluate(X_test, Y_test, verbose=1)
# print(f'Test results - Loss: {test_results[0]} - Accuracy: {test_results[1]}%')

Feature shape: (784,)
Train on 48000 samples, validate on 12000 samples
48000/48000 [==============================] - 2s 32us/sample - loss: 1.0111 - accuracy: 0.7331 - val_loss: 0.3708 - val_accuracy: 0.8951


In [6]:
# Convert into ONNX Format

from tensorflow.keras.models import load_model
from keras2onnx import convert_keras
import onnxruntime as rt
import numpy as np

model = load_model('model/DNN_MLP_basic.h5')

# keras model -> onnx 로 변환
onx = convert_keras(model, 'model/DNN_MLP_basic.onnx')

with open("model/DNN_MLP_basic.onnx", "wb") as f:
    f.write(onx.SerializeToString())

print('onnx로 변환 완료')

The maximum opset needed by this model is only 9.


onnx로 변환 완료


In [7]:
# X_test 10장만 테스트
X_test = X_test[0:10]
X_test.shape

(10, 784)

## Inference time 비교

In [8]:
# lenet - mnist
import time

# 시간측정
start = time.time()
# lenet - keras 
for i in range(10):
    pred_keras = model.predict(X_test)
end = time.time()
print('keras - DNN 걸린시간:', np.double(end-start))

keras - DNN 걸린시간: 0.3484160900115967


In [10]:
# Compute the prediction with ONNX Runtime
sess = rt.InferenceSession('model/DNN_MLP_basic.onnx')
input_name = sess.get_inputs()[0].name
label_name = sess.get_outputs()[0].name

# 시간측정
start = time.time()

# lenet - onnxruntime 
for i in range(10):
    pred_onx = sess.run([label_name], {input_name: X_test.astype(np.float32)})[0]
end = time.time()

print('onnxruntime DNN 걸린시간:', np.double(end-start))

onnxruntime DNN 걸린시간: 0.003989696502685547
